In [1]:
import numpy as np
import scanreader
import tifffile
from pathlib import Path
from mbo_utilities import get_metadata

In [2]:
from skimage.restoration import estimate_sigma

def estimate_noise_sk(img):
    return estimate_sigma(img, average_sigmas=True)

In [13]:
raw_data = Path("E://datasets/relaxed_resolution")
files = list(raw_data.glob("*.tif*"))
files = [str(file) for file in files]
files

['E:\\datasets\\relaxed_resolution\\mh89_2mm_FOV_50_550um_depth_250mW_som_stimuli_9min_00001_00001.tif',
 'E:\\datasets\\relaxed_resolution\\mh89_2mm_FOV_50_550um_depth_250mW_som_stimuli_9min_00001_00002.tif',
 'E:\\datasets\\relaxed_resolution\\mh89_2mm_FOV_50_550um_depth_250mW_som_stimuli_9min_00001_00003.tif']

In [6]:
metadata = get_metadata(files[0])
metadata

{'num_planes': 30,
 'num_frames': 1162,
 'fov': (400, 400.0),
 'num_rois': 5,
 'frame_rate': 6.4464,
 'pixel_resolution': array([2.78, 2.78]),
 'ndim': 4,
 'dtype': 'uint16',
 'size': 56167826400,
 'raw_height': 3704,
 'raw_width': 145,
 'tiff_pages': 34860,
 'roi_width_px': 144,
 'roi_height_px': 720,
 'sample_format': 'int16',
 'objective_resolution': 157.5}

In [14]:
scan = scanreader.read_scan(files, join_contiguous=True)

In [5]:
%%timeit

#### f, Y, X, Z, T
data = scan[:, :, :, 1, :].squeeze()
clone = np.zeros(

14.5 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
scan[:, :, :, 2, 1].squeeze()

5.82 ms ± 67.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
data = scan[:, :, :, 1, :].squeeze()
data.shape

(720, 720, 3486)

In [8]:
%%timeit

mean = data.mean(axis=1)
mean

862 ms ± 31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
import zarr

In [9]:
file_t = tifffile.imread(files[0], aszarr=True)

In [7]:
def get_zt(z, t):
    return scan[:, :, :, z, t].squeeze()

In [ ]:
mean_imgs = {}
import time
start = time.time()
for i in range(scan.num_channels):
    data = scan[:, :, :, i, :].squeeze()
    mean_imgs[i] = (data.mean(axis=-1), estimate_noise_sk(data[..., 0]))
print(f'Duration: {time.time() - start:.2f}')

In [11]:
import fastplotlib as fpl

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Available devices:
✅ (default) | NVIDIA RTX A4000 | DiscreteGPU | Vulkan | 560.94
✅ | NVIDIA RTX A4000 | DiscreteGPU | D3D12 | 
❗ | Microsoft Basic Render Driver | CPU | D3D12 | 
❗ | NVIDIA RTX A4000/PCIe/SSE2 | Unknown | OpenGL | 


In [16]:
import dask.array as da

In [11]:
import fastplotlib as fpl

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Available devices:
✅ (default) | NVIDIA RTX A4000 | DiscreteGPU | Vulkan | 560.94
✅ | NVIDIA RTX A4000 | DiscreteGPU | D3D12 | 
❗ | Microsoft Basic Render Driver | CPU | D3D12 | 
❗ | NVIDIA RTX A4000/PCIe/SSE2 | Unknown | OpenGL | 


In [16]:
iw = fpl.ImageWidget(scan, histogram_widget=False)
iw.show()

TypeError: `data` must be an array-like type or a list of array-like.You have passed the following type <class 'scanreader.scans.ScanMultiROI'>

In [14]:
iw = fpl.ImageWidget(np.moveaxis(data, -1, 0))
iw.window_funcs = {"t": (np.mean, 13)}
iw.show()

RFBOutputContext()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

In [69]:
import math
from scipy.signal import convolve2d

def estimate_noise(I):

  H, W = I.shape

  M = [[1, -2, 1],
       [-2, 4, -2],
       [1, -2, 1]]

  sigma = np.sum(np.sum(np.absolute(convolve2d(I, M))))
  sigma = sigma * math.sqrt(0.5 * math.pi) / (6 * (W-2) * (H-2))

  return sigma

In [70]:
sig = estimate_noise(data[..., 0])
sig

166.73132934796297

In [67]:
noise = estimate_noise_sk(data[..., 0])
noise

152.3732865925228

In [2]:
from __future__ import annotations

import argparse
from pathlib import Path

import numpy as np
import fastplotlib as fpl
from pprint import pprint
from mbo_utilities.lcp_io import get_metadata
from mbo_utilities.scanreader.exceptions import PathnameError, FieldDimensionMismatch
from mbo_utilities.scanreader.core import scans, expand_wildcard


def read_scan(pathnames, dtype=np.int16, join_contiguous=False):
    """ Reads a ScanImage scan. """
    # Expand wildcards
    filenames = expand_wildcard(pathnames)
    if len(filenames) == 0:
        error_msg = 'Pathname(s) {} do not match any files in disk.'.format(pathnames)
        raise PathnameError(error_msg)

    scan = ScanMultiROIReordered(join_contiguous=join_contiguous)

    # Read metadata and data (lazy operation)
    scan.read_data(filenames, dtype=dtype)

    return scan


class ScanMultiROIReordered(scans.ScanMultiROI):
    """
    A subclass of ScanMultiROI that ignores the num_fields dimension
    and reorders the output to [time, z, x, y].
    """

    def __getitem__(self, key):
        if not isinstance(key, tuple):
            key = (key,)

        # Call the parent class's __getitem__ with the reordered key
        item = super().__getitem__((0, key[2], key[3], key[1], key[0]))
        if item.ndim == 2:
            return item
        elif item.ndim == 3:
            return np.transpose(item, (2, 0, 1))
        else:
            raise FieldDimensionMismatch('ScanMultiROIReordered.__getitem__')

    @property
    def shape(self):
        return self.num_frames, self.num_channels, self.field_heights[0], self.field_widths[0]

    @property
    def ndim(self):
        return 4


def add_args(parser: argparse.ArgumentParser):
    """
    Add command-line arguments to the parser, dynamically adding arguments
    for each key in the `ops` dictionary.

    Parameters
    ----------
    parser : argparse.ArgumentParser
        The argument parser to which arguments are added.

    Returns
    -------
    argparse.ArgumentParser
        The parser with added arguments.
    """
    parser.add_argument('--path', type=str,
                        help='Path to a directory containing raw scanimage tiff files for a single session.')
    parser.add_argument('--version', action='store_true', help='Print the version of the package.')
    return parser


def main():
    parser = argparse.ArgumentParser(description="Preview a scanimage imaging session.")
    parser = add_args(parser)
    args = parser.parse_args()

    # Handle version
    if args.version:
        import mbo_utilities as mbo
        print("lbm_caiman_python v{}".format(mbo.__version__))
        return

    files = [str(f) for f in Path(args.path).expanduser().glob('*.tif*')]
    metadata = get_metadata(files[0])
    pprint(metadata)
    scan = read_scan(files, join_contiguous=True)
    iw = fpl.ImageWidget(scan, histogram_widget=False)
    iw.show()
    if fpl.__version__ == "0.2.0":
        fpl.run()
    elif fpl.__version__ == "0.3.0":
        fpl.loop.run()

In [8]:
path = Path(r"D:\DATA\2025-01-16_GCaMP8s_tdtomato_mk302")
files = [str(f) for f in Path(path).expanduser().glob('*.tif*')]
files[0]

'D:\\DATA\\2025-01-16_GCaMP8s_tdtomato_mk302\\mk302_2umpx_17p07hz_224pxby448px_2mroi_200mw_50to550umdeep_GCaMP8s_00001_00001.tif'

In [6]:
metadata = get_metadata(files[0])

In [11]:
scan = read_scan(files[0], join_contiguous=True)

TypeError: 'float' object cannot be interpreted as an integer